In [3]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from models import Base
from utils.general import generate_uppercase_alphabets
from utils.create import add_well_type_to_plate_type, add_lib_well_to_plate, add_xtal_wells_to_plate, transfer_xtal_to_pin
from utils.read import get_or_create, get_xtal_well

engine = create_engine("sqlite:///../test/test.db")
Session = sessionmaker(bind=engine)
session = Session()

Base.metadata.create_all(engine)

In [4]:
from models import (
    Base,
    Project,
    LibraryPlateType,
    LibraryWellType,
    LibraryPlate,
    LibraryWell,
    XtalPlate,
    XtalPlateType,
    XtalWellType,
    XtalWell,
    WellMap,
    Batch,
    DropPosition,
    EchoTransfer,
    XrayStatusEnum,
    PuckType,
    Puck,
    Pin,
)
import pandas
from sqlalchemy.exc import NoResultFound
DATABASE_SETUP_ONLY = True

## Add Project

In [5]:
project = Project(target="test", year=2023, cycle=3, visit=1)
session.add(project)

## Add Library Plate type

### 1536 plate

In [6]:
LIB_PLATE_ROWS_1536 = 32
LIB_PLATE_COLS_1536 = 48
lib_plate_type_1536 = LibraryPlateType(name="1536LDV", rows=LIB_PLATE_ROWS_1536, columns=LIB_PLATE_COLS_1536)
session.add(lib_plate_type_1536)
a_to_af = generate_uppercase_alphabets(LIB_PLATE_ROWS_1536)
lib_well_type_names_1536 = [
        {"name": f"{i}{str(j).zfill(2)}"} for i in a_to_af for j in range(1, LIB_PLATE_COLS_1536+1)
    ]
add_well_type_to_plate_type(session, lib_well_type_names_1536, lib_plate_type_1536)

### Create library plate and add associate chemicals with it

In [7]:
lib_plate_dsip = LibraryPlate(library_plate_type=lib_plate_type_1536, name="DSI-poised")
session.add(lib_plate_type_1536)
session.add(lib_plate_dsip)
session.commit()

# Read the vendor csv and populate the library plate
df_dsip = pandas.read_csv("../test/dsip.csv")
add_lib_well_to_plate(session, df_dsip, lib_plate_dsip)

### 384 plate

In [8]:
LIB_PLATE_ROWS_384 = 16
LIB_PLATE_COLS_384 = 24
lib_plate_type_384 = LibraryPlateType(
    name="384PP",
    rows=LIB_PLATE_ROWS_384,
    columns=LIB_PLATE_COLS_384
)
session.add(lib_plate_type_384)
a_to_p = generate_uppercase_alphabets(LIB_PLATE_ROWS_384)
lib_well_type_names_384 = [
        {"name": f"{i}{str(j).zfill(2)}"} for i in a_to_p for j in range(1, LIB_PLATE_COLS_384+1)
    ]
add_well_type_to_plate_type(session, lib_well_type_names_384, lib_plate_type_384)

### LifeChemicals fragment diversity set #3

In [9]:
lib_plate_fds3 = LibraryPlate(
    library_plate_type=lib_plate_type_384,
    name="lc-fds-III"
)
session.add(lib_plate_fds3)
session.commit()

# Read the vendor csv and populate the library plate
df_fds3 = pandas.read_csv("../test/lc-fds-III.csv")
print(df_fds3)
add_lib_well_to_plate(session, df_fds3, lib_plate_fds3)

     catalog_id                            smiles  \
0    F6675-8150     CN(CC(F)(F)F)C1CN(c2cnccn2)C1   
1    F6616-1270           Br.c1cc(SCC2CCOCC2)ccn1   
2    F6608-0984           CC(Oc1nccn(C)c1=O)C1CC1   
3    F6608-1013          CN1CCCC(COc2cnccc2C#N)C1   
4    F6541-4519          OCC1CCCN1c1ccc(C2CC2)nn1   
..          ...                               ...   
315  F1913-0560           Cl.O=C(O)C1(N2CCCC2)CC1   
316  F1913-0654    Cl.FC(F)(F)c1cccc(C2=CCNCC2)n1   
317  F2125-0006         Nc1cccc(-c2cn3cccnc3n2)c1   
318  F2145-0837       N#CCc1nc(-c2cc3ccccc3o2)cs1   
319  F2113-0652  Nc1nc2cc(C(F)(F)F)ccc2n1Cc1ccco1   

                                         chemical_name  price_coeff  \
0    N-methyl-1-(pyrazin-2-yl)-N-(2,2,2-trifluoroet...          1.5   
1    4-{[(oxan-4-yl)methyl]sulfanyl}pyridine hydrob...          1.0   
2    3-(1-cyclopropylethoxy)-1-methyl-1,2-dihydropy...          1.5   
3    3-[(1-methylpiperidin-3-yl)methoxy]pyridine-4-...          1.0   
4    [1-

### Solvent test plate

In [10]:
SOLVENT_WELL="G07"
lib_plate_solvent = LibraryPlate(
    library_plate_type=lib_plate_type_384,
    name=f"solvent_{SOLVENT_WELL}"
)
session.add(lib_plate_solvent)
session.commit()

# single well in solvent plate
df_solvent = pandas.DataFrame(
    {"catalog_id":["DMSO"],"smiles":["CS(C)=O"],"well":[SOLVENT_WELL]}
)

# populate the solvent library plate
add_lib_well_to_plate(session, df_solvent, lib_plate_solvent)

### Create Xtal plate type and add an echo to shifter mapping to the plate type

### SwissCI-MRC-2d

In [11]:
# Create and add xtal plate
xtal_plate_type = XtalPlateType(name="SwissCI-MRC-2d")
session.add(xtal_plate_type)

# Create lists that map echo wells to shifter wells 
a_to_h = generate_uppercase_alphabets(8)
a_to_p = generate_uppercase_alphabets(16)
echo = [f"{i}{j}" for i in a_to_p for j in range(1, 13)]
shifter = [f"{i}{k}{j}" for i in a_to_h for j in ["a", "b"] for k in range(1, 13)]
plate_maps = [{"echo": i, "shifter": j} for i, j in zip(echo, shifter)]

# Loop through plate map list, if the shifter well name ends with a "b", add a y offset
for plate_map in plate_maps:
    x_offset = 0
    if plate_map["shifter"][-1] == "b":
        y_offset = 1350  # microns
    else:
        y_offset = 0
    well_map = WellMap(well_pos_x=x_offset, well_pos_y=y_offset, **plate_map)
    session.add(well_map)
    xtal_well_type = XtalWellType(name=plate_map["shifter"], well_map=well_map)
    session.add(xtal_well_type)


### SwissCI-MRC-3d

In [18]:
# Create and add xtal plate
xtal_plate_type = XtalPlateType(name="SwissCI-MRC-3d")
session.add(xtal_plate_type)

# Create lists that map echo wells to shifter wells 
a_to_h = generate_uppercase_alphabets(8)
a_to_p = generate_uppercase_alphabets(16)
echo = [f"{i}{j}" for i in a_to_p for j in range(1, 25)]
shifter = []
for i in a_to_h:
    for j,k in zip([item for item in range(1,13) for _ in range(2)], 12*["a","b"]):
        shifter.append(f"{i}{j}{k}")
    for j,k in zip([item for item in range(1,13) for _ in range(2)], 12*["c","d"]):
        shifter.append(f"{i}{j}{k}")

plate_maps = [{"echo": i, "shifter": j} for i, j in zip(echo, shifter)]

# Loop through plate map list, if the shifter well name ends with a "b" or "d",
# add x_offset. This labeling corresponds to even numbered columns in echo format.
for plate_map in plate_maps:
    y_offset = 0
    if plate_map["shifter"][-1] in ["b","d"]:
        x_offset = -600  # microns
    else:
        x_offset = 0
    well_map = WellMap(well_pos_x=x_offset, well_pos_y=y_offset, **plate_map)
    session.add(well_map)
    xtal_well_type = XtalWellType(name=plate_map["shifter"], well_map=well_map)
    session.add(xtal_well_type)

### Add drop position codes and offsets to the database

In [13]:
drop_position_codes = [
    {"name": "ul", "x_offset": -300, "y_offset": 300},
    {"name": "lu", "x_offset": -300, "y_offset": 300},
    {"name": "u", "x_offset": 0, "y_offset": 300},
    {"name": "ru", "x_offset": 300, "y_offset": 300},
    {"name": "ur", "x_offset": 300, "y_offset": 300},
    {"name": "l", "x_offset": -300, "y_offset": 0},
    {"name": "c", "x_offset": 0, "y_offset": 0},
    {"name": "r", "x_offset": 300, "y_offset": 0},
    {"name": "dl", "x_offset": -300, "y_offset": -300},
    {"name": "ld", "x_offset": -300, "y_offset": -300},
    {"name": "dr", "x_offset": 300, "y_offset": -300},
    {"name": "rd", "x_offset": 300, "y_offset": -300},
    {"name": "d", "x_offset": -300, "y_offset": 0},
]

session.add_all(
    [
        DropPosition(**drop_position_code)
        for drop_position_code in drop_position_codes
    ]
)

### The rest of the steps described here are used for reference. It shows code that populates the database during an actual experimental run. It is not part of the database setup

#### Steps to add experiment info to database: 
- Use the imaging csv from the shifter to create an instance of an Xtal plate

In [14]:
if not DATABASE_SETUP_ONLY:
    imaging_df = pandas.read_csv("../test/imaging.csv", skiprows=8)
    # shifter app leaves semi-colons in front of all header text, fix that
    # add some light spreadsheet validation (unique plate_ids)
    imaging_df.rename(columns={";PlateType": "PlateType"}, inplace=True)
    unique_plate_ids = imaging_df["PlateID"].unique()
    if len(unique_plate_ids) != 1:
        raise ValueError("inconsistent plateIDs; plateIDs should all be the same")

    xtal_plate = XtalPlate(plate_type=xtal_plate_type, name=unique_plate_ids[0])
    session.add(xtal_plate)
    add_xtal_wells_to_plate(session, imaging_df, xtal_plate)

    # create a batch
    batch = Batch()
    session.add(batch)
    # populate transfers
    # need to ensure that we go row by row through xtal plate, column
    # by column through library plate to minimize xy motions during transfer
    # added sequence column to well tables to enforce this ordering
    xtal_well_query = (
        session.query(XtalWell)
        .order_by(XtalWell.sequence)
        .filter(XtalWell.plate.has(name="pmtest"))
    )

    library_well_query = (
        session.query(LibraryWell)
        .order_by(LibraryWell.sequence)
        .filter(LibraryWell.plate.has(name="DSI-poised"))
    )

    for xtal_well, library_well in zip(xtal_well_query, library_well_query):
        transfer = EchoTransfer(
            batch=batch, from_well=library_well, to_well=xtal_well, transfer_volume=25
        )
        session.add(transfer)

    # prepare to ingest shifter harvesting csv
    # make sure there are pucks (and puck_types)
    puck_names = [{"name": name} for name in ["testpuck", "FGZ001", "FGZ002"]]
    for puck_name in puck_names:
        get_or_create(session, PuckType, **puck_name)

    harvesting_df = pandas.read_csv("../test/harvesting.csv", skiprows=8)
    harvesting_df.rename(columns={";PlateType": "PlateType"}, inplace=True)

    puck_data = [
        {"puck_type": session.query(PuckType).filter_by(name=k).one()}
        for k in harvesting_df["DestinationName"].dropna().unique()
    ]
    for puck in puck_data:
        get_or_create(session, Puck, **puck)

    for index, row in harvesting_df.iterrows():
        # this entry will be populated if something happened at the well,
        # successful or not
        if pandas.notna(row["Comment"]):
            shifter_well_pos = (
                f"{row['PlateRow']}{row['PlateColumn']}{row['PositionSubWell']}"
            )
            xtal_well = get_xtal_well(session, row["PlateID"], shifter_well_pos)
            xtal_well.harvest_comment = row["Comment"]
            xtal_well.harvesting_status = True
            xtal_well.time_arrival = pandas.to_datetime(
                row["TimeArrival"], format="%d/%m/%Y %H:%M:%S"
            )
            echo_transfer = (
                session.query(EchoTransfer)
                .filter(EchoTransfer.to_well_id == xtal_well.uid)
                .first()
            )
            if echo_transfer:
                library_well = echo_transfer.from_well
                library_well.used = True

            # this entry will only be populated if the pin made it into the puck
            if pandas.notna(row["DestinationLocation"]):
                transfer_xtal_to_pin(session, xtal_well, row["DestinationName"], 
                                    row["DestinationLocation"], row["TimeDeparture"])

In [15]:
len(session.query(XtalWell).filter(
            XtalWell.plate.has(name="pmtest"),
            #XtalWell.well_type.has(name=shifter_well_pos),
        ).all())

0

In [16]:
session.query(XtalWellType).filter_by(name="A1a").one()

MultipleResultsFound: Multiple rows were found when exactly one was required

In [19]:
session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: xtal_plate_type.name
[SQL: INSERT INTO xtal_plate_type (name) VALUES (?)]
[parameters: ('SwissCI-MRC-3d',)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)